In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

# Paths to the zip files and where you want to unzip them
png_images = '/content/drive/MyDrive/png-images.zip'  # Update with the path to your first zip file
healthy = '/content/drive/MyDrive/new-20241016T105852Z-001.zip'  # Update with the path to your second zip file

output_dir_1 = '/content/drive/MyDrive/'  # Folder where you want to unzip the first file
output_dir_2 = '/content/drive/MyDrive/'  # Folder where you want to unzip the second file

# Function to unzip a file
def unzip_file(zip_file, output_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f'Unzipped {zip_file} to {output_dir}')

# Unzipping the first folder
unzip_file(png_images, output_dir_1)

# Unzipping the second folder
unzip_file(healthy, output_dir_2)


Unzipped /content/drive/MyDrive/png-images.zip to /content/drive/MyDrive/
Unzipped /content/drive/MyDrive/new-20241016T105852Z-001.zip to /content/drive/MyDrive/


In [3]:
import os
import shutil

# Paths to the original lesion and synthetic lesion directories
original_lesion_dir = '/content/drive/MyDrive/png-images'
synthetic_lesion_dir = '/content/drive/MyDrive/1729016248.5377333'
combined_lesion_dir = '/content/drive/MyDrive/combined-lesion'

# Create the combined lesion directory if it doesn't exist
if not os.path.exists(combined_lesion_dir):
    os.makedirs(combined_lesion_dir)

# Copy original lesion images to the combined folder
for filename in os.listdir(original_lesion_dir):
    file_path = os.path.join(original_lesion_dir, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, combined_lesion_dir)

# Copy synthetic lesion images to the combined folder
for filename in os.listdir(synthetic_lesion_dir):
    file_path = os.path.join(synthetic_lesion_dir, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, combined_lesion_dir)

print("Datasets combined successfully into:", combined_lesion_dir)

Datasets combined successfully into: /content/drive/MyDrive/combined-lesion


In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Set up paths for your data
combined_lesion_dir = '/content/drive/MyDrive/combined-lesion'  # Path for combined synthetic and original lesion images
healthy_dir = '/content/drive/MyDrive/new'  # Path for healthy images

# Check if directories exist
assert os.path.exists(combined_lesion_dir), "Combined lesion data directory does not exist!"
assert os.path.exists(healthy_dir), "Healthy data directory does not exist!"

# Data augmentation settings
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% validation split
)

# Training and validation data generators
train_generator = datagen.flow_from_directory(
    directory='/content/drive/MyDrive',
    classes=['combined-lesion', 'new'],
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    directory='/content/drive/MyDrive',
    classes=['combined-lesion', 'new'],
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Save the final trained model
model.save('/content/drive/MyDrive/final_cnn_model.h5')

Found 1208 images belonging to 2 classes.
Found 302 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 320s 4s/step - accuracy: 0.7076 - loss: 0.5428 - val_accuracy: 0.8368 - val_loss: 0.4331
Epoch 2/10
 1/75 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.8750 - loss: 0.3100

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.8750 - loss: 0.3100 - val_accuracy: 0.8571 - val_loss: 0.3939
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 352s 4s/step - accuracy: 0.9032 - loss: 0.3025 - val_accuracy: 0.9549 - val_loss: 0.1680
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - accuracy: 1.0000 - loss: 0.0744 - val_accuracy: 0.9286 - val_loss: 0.1031
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 287s 4s/step - accuracy: 0.9455 - loss: 0.1575 - val_accuracy: 0.9549 - val_loss: 0.1135
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 1.0000 - loss: 0.0733 - val_accuracy: 0.9286 - val_loss: 0.0866
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 325s 4s/step - accuracy: 0.9696 - loss: 0.1117 - val_accuracy: 0.9826 - val_loss: 0.0366
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 1.0000 - loss: 0.0824 - val_accuracy: 1.0000 - val_loss: 0.0083
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 309s 4s/step - accuracy: 0.9824 - loss: 0.0592 - val_accuracy: 0.9826 - val_loss: 0.

Validation Accuracy: 98.34%
